In [7]:
import pandas as pd

# Lee el archivo CSV comprimido en un DataFrame
df = pd.read_csv("D:/Documentos/Estudio/Maestria/[01] Maestria Cs Datos/DMEF/DMEyF/datasets_competencia_02_sin_baja_final.csv.gz", compression='gzip')



C:\Users\LeonardoIula\AppData\Local\Temp\ipykernel_27208\3511240054.py:4: DtypeWarning: Columns (154) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("D:/Documentos/Estudio/Maestria/[01] Maestria Cs Datos/DMEF/DMEyF/datasets_competencia_02_sin_baja_final.csv.gz", compression='gzip')


In [15]:

# Contar la cantidad de meses únicos (foto_mes) por cliente
clientes_por_meses = df.groupby('numero_de_cliente')['foto_mes'].nunique().reset_index(name='cantidad_meses')

# Filtrar clientes que aparecen en 1, 2, 3 o 4 meses y contar cuántos hay en cada grupo
conteo_por_meses = clientes_por_meses['cantidad_meses'].value_counts().sort_index()

# Mostrar resultados
print(conteo_por_meses)

# Si deseas ver solo los que aparecen en 1, 2, 3 o 4 meses:
conteo_por_meses = conteo_por_meses.loc[conteo_por_meses.index.isin([1, 2, 3])]
print(conteo_por_meses)


1       2141
2       1943
3       2271
4       2184
5       2171
6       2527
7       2047
8       1820
9       1931
10      2293
11      2602
12      2474
13      2443
14      2870
15      3373
16      2286
17      1377
18      3208
19      4010
20      4216
21      2972
22      2801
23      3244
24      2611
25      2879
26      2735
27      2582
28      2047
29      1713
30      1272
31      2184
32    107909
Name: cantidad_meses, dtype: int64
1    2141
2    1943
3    2271
Name: cantidad_meses, dtype: int64


In [16]:
def calcular_clientes_nuevos(df, col_cliente='numero_de_cliente', col_mes='foto_mes'):
    """
    Calcula cuántos clientes son nuevos en cada mes en relación al mes anterior.

    Parámetros:
        - df: DataFrame con los datos.
        - col_cliente: Nombre de la columna que identifica a los clientes. (Por defecto 'numero_de_cliente')
        - col_mes: Nombre de la columna que identifica los meses. (Por defecto 'foto_mes')

    Retorna:
        - DataFrame con dos columnas: 'foto_mes' y 'clientes_nuevos'.
    """
    # Asegurar que los datos están ordenados por mes
    df = df.sort_values(by=[col_mes, col_cliente])
    
    # Obtener los meses únicos ordenados
    meses = sorted(df[col_mes].unique())
    
    # Inicializar diccionario para guardar los resultados
    clientes_nuevos_por_mes = {}
    
    # Inicializar conjunto para el seguimiento de clientes históricos
    clientes_historicos = set()
    
    for mes in meses:
        # Filtrar clientes únicos del mes actual
        clientes_actuales = set(df[df[col_mes] == mes][col_cliente])
        
        # Determinar nuevos clientes (los que no están en el histórico)
        nuevos_clientes = clientes_actuales - clientes_historicos
        
        # Guardar el conteo de nuevos clientes para el mes actual
        clientes_nuevos_por_mes[mes] = len(nuevos_clientes)
        
        # Actualizar el histórico agregando los clientes actuales
        clientes_historicos.update(clientes_actuales)
    
    # Convertir resultados a DataFrame
    resultado = pd.DataFrame(list(clientes_nuevos_por_mes.items()), columns=['foto_mes', 'clientes_nuevos'])
    
    return resultado


In [17]:

# Calcular clientes nuevos por mes
nuevos_por_mes = calcular_clientes_nuevos(df)

# Imprimir los resultados
print(nuevos_por_mes)

# Si quieres guardar el resultado
nuevos_por_mes.to_csv("clientes_nuevos_por_mes.csv", index=False)


    foto_mes  clientes_nuevos
0     201901           124064
1     201902             1731
2     201903             1358
3     201904             1245
4     201905             1422
5     201906             2101
6     201907             2195
7     201908             2545
8     201909             2332
9     201910             2904
10    201911             2558
11    201912             2609
12    202001             4028
13    202002             3733
14    202003             2752
15    202004              701
16    202005             1755
17    202006             3016
18    202007             2627
19    202008             1904
20    202009             1845
21    202010             1825
22    202011             1565
23    202012             1264
24    202101             1139
25    202102             1242
26    202103             1814
27    202104             1412
28    202105             1504
29    202106             1429
30    202107             1172
31    202108             1345


In [5]:
# Excluir los últimos 3 meses del análisis
meses_a_excluir = sorted(df['foto_mes'].unique())[-3:]
df_excluido = df[~df['foto_mes'].isin(meses_a_excluir)]

# Ordenar el DataFrame por cliente y mes
df_sorted = df_excluido.sort_values(by=['numero_de_cliente', 'foto_mes'])

# Calcular diferencias consecutivas de 'foto_mes' por cliente
df_sorted['mes_diff'] = df_sorted.groupby('numero_de_cliente')['foto_mes'].diff()

# Identificar un nuevo bloque cuando el mes no es consecutivo
df_sorted['es_nuevo_bloque'] = (df_sorted['mes_diff'] != 1).astype(int)

# Crear un identificador único para bloques consecutivos por cliente
df_sorted['grupo_bloques'] = df_sorted.groupby('numero_de_cliente')['es_nuevo_bloque'].cumsum()

# Contar los meses consecutivos en cada bloque
bloques = df_sorted.groupby(['numero_de_cliente', 'grupo_bloques']).size().reset_index(name='meses_consecutivos')

# Obtener el máximo de meses consecutivos por cliente
max_meses_consecutivos = bloques.groupby('numero_de_cliente')['meses_consecutivos'].max().reset_index()

# Identificar clientes con menos de 4 meses consecutivos
clientes_vida_corta = max_meses_consecutivos[max_meses_consecutivos['meses_consecutivos'] < 4]['numero_de_cliente']

# Verificar la cantidad de clientes de vida corta por mes
df_vida_corta = df[df['numero_de_cliente'].isin(clientes_vida_corta)]
conteo_por_mes = df_vida_corta.groupby('foto_mes')['numero_de_cliente'].nunique()

# Imprimir los resultados
print("Clientes de vida corta por mes:")
print(conteo_por_mes)

# Crear un nuevo dataset sin clientes de vida corta
df_filtrado = df[~df['numero_de_cliente'].isin(clientes_vida_corta)]

# Verificar control: comparación de tamaños
clientes_originales = df['numero_de_cliente'].nunique()
clientes_filtrados = df_filtrado['numero_de_cliente'].nunique()

print(f"Clientes originales: {clientes_originales}")
print(f"Clientes después de eliminar vida corta: {clientes_filtrados}")




Clientes de vida corta por mes:
foto_mes
201901    2789
201902    2127
201903     747
201904      12
201905      18
201906      23
201907      24
201908      31
201909      28
201910      33
201911      43
201912      51
202001      45
202002      36
202003      50
202004      24
202005      22
202006      22
202007      32
202008      30
202009      35
202010      50
202011      49
202012      53
202101      58
202102      52
202103    1839
202104    3226
202105    4718
202106    4720
202107    4705
202108    4700
Name: numero_de_cliente, dtype: int64
Clientes originales: 185811
Clientes después de eliminar vida corta: 177959


In [6]:
# Guardar el dataset filtrado si es necesario
df_filtrado.to_csv("dataset_filtrado_sin_vida_corta.csv", index=False)

In [7]:
# Filtrar los clientes de vida corta del DataFrame original
df_vida_corta = df[df['numero_de_cliente'].isin(clientes_vida_corta)]

# Calcular la distribución de 'clase_ternaria' por mes para los clientes de vida corta
distribucion_clase_ternaria = df_vida_corta.groupby(['foto_mes', 'clase_ternaria'])['numero_de_cliente'].nunique().unstack(fill_value=0)

# Calcular la proporción de cada clase dentro de cada mes
proporcion_clase_ternaria = distribucion_clase_ternaria.div(distribucion_clase_ternaria.sum(axis=1), axis=0)

# Imprimir los resultados
print("Proporción de 'clase_ternaria' para los clientes de vida corta por mes:")
print(proporcion_clase_ternaria)

# Si prefieres ver la distribución en formato más amigable:
distribucion_clase_ternaria_with_proportions = distribucion_clase_ternaria.copy()
distribucion_clase_ternaria_with_proportions['Proporción_Total'] = distribucion_clase_ternaria.sum(axis=1)
distribucion_clase_ternaria_with_proportions = distribucion_clase_ternaria_with_proportions.join(proporcion_clase_ternaria, rsuffix='_Proporcion')

print("\nDistribución y proporción total de 'clase_ternaria' por mes:")
print(distribucion_clase_ternaria_with_proportions)


Proporción de 'clase_ternaria' para los clientes de vida corta por mes:
clase_ternaria    BAJA+1    BAJA+2  CONTINUA
foto_mes                                    
201901          0.241664  0.253137  0.505199
201902          0.333803  0.317819  0.348378
201903          0.000000  0.991968  0.008032
201904          0.000000  0.333333  0.666667
201905          0.000000  0.444444  0.555556
201906          0.000000  0.521739  0.478261
201907          0.000000  0.250000  0.750000
201908          0.000000  0.516129  0.483871
201909          0.000000  0.678571  0.321429
201910          0.000000  0.272727  0.727273
201911          0.000000  0.162791  0.837209
201912          0.000000  0.294118  0.705882
202001          0.000000  0.466667  0.533333
202002          0.000000  0.194444  0.805556
202003          0.000000  0.560000  0.440000
202004          0.000000  0.333333  0.666667
202005          0.000000  0.772727  0.227273
202006          0.000000  0.409091  0.590909
202007          0.000000  0.

In [18]:
# Asegúrate de que el DataFrame esté ordenado por cliente y mes
df_sorted = df.sort_values(by=['numero_de_cliente', 'foto_mes'])

# Calcular las diferencias consecutivas de 'foto_mes' por cliente
df_sorted['mes_diff'] = df_sorted.groupby('numero_de_cliente')['foto_mes'].diff()

# Identificar un nuevo bloque cuando el mes no es consecutivo
df_sorted['es_nuevo_bloque'] = (df_sorted['mes_diff'] != 1).astype(int)

# Crear un identificador único para bloques consecutivos por cliente
df_sorted['grupo_bloques'] = df_sorted.groupby('numero_de_cliente')['es_nuevo_bloque'].cumsum()

# Contar los meses consecutivos en cada bloque
bloques = df_sorted.groupby(['numero_de_cliente', 'grupo_bloques']).size().reset_index(name='meses_consecutivos')

# Obtener el máximo de meses consecutivos por cliente
max_meses_consecutivos = bloques.groupby('numero_de_cliente')['meses_consecutivos'].max().reset_index()

# Identificar clientes con 2 meses consecutivos, excluyendo los primeros 2 y los últimos 3 meses
# Obtener los primeros 2 y últimos 3 meses en el dataset
primeros_meses = df['foto_mes'].min()  # Primer mes
ultimos_meses = df['foto_mes'].max()  # Último mes
meses_excluir = set(range(primeros_meses, primeros_meses + 2)).union(set(range(ultimos_meses - 2, ultimos_meses + 1)))

# Excluir los clientes con 2 meses consecutivos, excepto en los primeros 2 y últimos 3 meses
clientes_vida_corta = max_meses_consecutivos[(max_meses_consecutivos['meses_consecutivos'] == 2) & 
                                             (~max_meses_consecutivos['numero_de_cliente'].isin(
                                                 df_sorted[df_sorted['foto_mes'].isin(meses_excluir)]['numero_de_cliente']
                                             ))]['numero_de_cliente']

# Filtrar el DataFrame eliminando a los clientes de vida corta
df_filtrado = df[~df['numero_de_cliente'].isin(clientes_vida_corta)]

# Verificar el tamaño antes y después de la eliminación
clientes_originales = df['numero_de_cliente'].nunique()
clientes_filtrados = df_filtrado['numero_de_cliente'].nunique()

print(f"Clientes originales: {clientes_originales}")
print(f"Clientes después de eliminar vida corta: {clientes_filtrados}")




Clientes originales: 185136
Clientes después de eliminar vida corta: 185027


In [19]:
# Asegúrate de que el DataFrame esté ordenado por cliente y mes
df_sorted = df.sort_values(by=['numero_de_cliente', 'foto_mes'])

# Calcular las diferencias consecutivas de 'foto_mes' por cliente
df_sorted['mes_diff'] = df_sorted.groupby('numero_de_cliente')['foto_mes'].diff()

# Identificar un nuevo bloque cuando el mes no es consecutivo
df_sorted['es_nuevo_bloque'] = (df_sorted['mes_diff'] != 1).astype(int)

# Crear un identificador único para bloques consecutivos por cliente
df_sorted['grupo_bloques'] = df_sorted.groupby('numero_de_cliente')['es_nuevo_bloque'].cumsum()

# Contar los meses consecutivos en cada bloque
bloques = df_sorted.groupby(['numero_de_cliente', 'grupo_bloques']).size().reset_index(name='meses_consecutivos')

# Obtener el máximo de meses consecutivos por cliente
max_meses_consecutivos = bloques.groupby('numero_de_cliente')['meses_consecutivos'].max().reset_index()

# Identificar clientes con 2 meses consecutivos, excluyendo los primeros 2 y los últimos 3 meses
# Obtener los primeros 2 y últimos 3 meses en el dataset
primeros_meses = df['foto_mes'].min()  # Primer mes
ultimos_meses = df['foto_mes'].max()  # Último mes
meses_excluir = set(range(primeros_meses, primeros_meses + 2)).union(set(range(ultimos_meses - 2, ultimos_meses + 1)))

# Excluir los clientes con 2 meses consecutivos, excepto en los primeros 2 y últimos 3 meses
clientes_vida_corta = max_meses_consecutivos[(max_meses_consecutivos['meses_consecutivos'] == 3) & 
                                             (~max_meses_consecutivos['numero_de_cliente'].isin(
                                                 df_sorted[df_sorted['foto_mes'].isin(meses_excluir)]['numero_de_cliente']
                                             ))]['numero_de_cliente']

# Filtrar el DataFrame eliminando a los clientes de vida corta
df_filtrado = df[~df['numero_de_cliente'].isin(clientes_vida_corta)]

# Verificar el tamaño antes y después de la eliminación
clientes_originales = df['numero_de_cliente'].nunique()
clientes_filtrados = df_filtrado['numero_de_cliente'].nunique()

print(f"Clientes originales: {clientes_originales}")
print(f"Clientes después de eliminar vida corta: {clientes_filtrados}")

Clientes originales: 185136
Clientes después de eliminar vida corta: 184986


In [20]:
# Guardar el dataset filtrado con compresión 'gzip'
df_filtrado.to_csv("dataset_filtrado_sin_vida_corta_2.csv.gz", index=False, compression='gzip')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4718251 entries, 0 to 4718250
Columns: 155 entries, numero_de_cliente to clase_ternaria
dtypes: float64(91), int64(63), object(1)
memory usage: 5.4+ GB


In [4]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4718019 entries, 0 to 4718250
Columns: 155 entries, numero_de_cliente to clase_ternaria
dtypes: float64(91), int64(63), object(1)
memory usage: 5.5+ GB
